# Simple Position Viewer for IB Gateway - Account DU9233079

Minimal notebook to retrieve current positions from Interactive Brokers Gateway.
Focuses on vertical spreads with the specific columns requested.

In [20]:
import time
import threading
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import random

In [21]:
class SimplePositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = {}
        self.market_data = {}
        self.close_prices = {}  # Store closing prices
        self.position_end = False
        self.next_req_id = 1
        self.req_id_to_contract = {}  # Map request IDs to contracts for historical data
        
    def error(self, reqId, errorCode, errorString, *args):
        if errorCode not in [2104, 2106, 2158]:  # Skip connection messages
            print(f"Error {errorCode}: {errorString}")
    
    def position(self, account, contract, position, avgCost):
        """Store position data for account DU9233079 only"""
        if account != "DU9233079" or position == 0:
            return
            
        key = f"{contract.symbol}_{contract.secType}_{contract.strike}_{contract.right}_{contract.lastTradeDateOrContractMonth}"
        
        self.positions[key] = {
            'symbol': contract.symbol,
            'secType': contract.secType,
            'strike': float(contract.strike) if contract.strike else 0,  # Convert to float
            'right': contract.right,
            'expiry': contract.lastTradeDateOrContractMonth,
            'position': position,
            'avgCost': float(avgCost) if avgCost else 0,  # Convert to float
            'contract': contract
        }
        
        # Request both live market data and historical closing data
        req_id = self.next_req_id
        self.next_req_id += 1
        self.reqMktData(req_id, contract, "", False, False, [])
        
        # Request historical data for closing price
        hist_req_id = self.next_req_id
        self.next_req_id += 1
        self.req_id_to_contract[hist_req_id] = key
        
        # Request 2 days of daily data to ensure we get the last close
        self.reqHistoricalData(
            hist_req_id, contract, "", "2 D", "1 day", "TRADES", 1, 1, False, []
        )
        
    def positionEnd(self):
        self.position_end = True
        print(f"Found {len(self.positions)} positions")
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Store current market prices"""
        if tickType == 4:  # Last price
            # Find which position this price belongs to
            req_num = reqId - 1  # Adjust for 0-based indexing
            if req_num < len(list(self.positions.keys())):
                key = list(self.positions.keys())[req_num]
                self.market_data[key] = float(price) if price else 0  # Convert to float
    
    def historicalData(self, reqId, bar):
        """Store historical closing prices"""
        if reqId in self.req_id_to_contract:
            key = self.req_id_to_contract[reqId]
            # Store the closing price from the bar
            self.close_prices[key] = float(bar.close) if bar.close else 0
    
    def historicalDataEnd(self, reqId, start, end):
        """Called when historical data request is complete"""
        if reqId in self.req_id_to_contract:
            key = self.req_id_to_contract[reqId]
            if key in self.close_prices:
                print(f"Got closing price for {key}: ${self.close_prices[key]:.2f}")
    
    def get_best_price(self, key):
        """Get the best available price: live market data first, then closing price"""
        # Try live market data first
        if key in self.market_data and self.market_data[key] > 0:
            return self.market_data[key], "live"
        
        # Fall back to closing price
        if key in self.close_prices and self.close_prices[key] > 0:
            return self.close_prices[key], "close"
        
        # No price available
        return None, "none"
    
    def create_spreads_dataframe(self):
        """Create DataFrame with the exact columns requested"""
        if not self.positions:
            return pd.DataFrame()
        
        # Group options by symbol and expiry to find spreads
        option_groups = {}
        for key, pos in self.positions.items():
            if pos['secType'] == 'OPT':
                group_key = f"{pos['symbol']}_{pos['expiry']}_{pos['right']}"
                if group_key not in option_groups:
                    option_groups[group_key] = []
                option_groups[group_key].append((key, pos))
        
        spread_data = []
        
        # Process each group to find vertical spreads
        for group_key, options in option_groups.items():
            if len(options) >= 2:
                # Sort by strike price
                options.sort(key=lambda x: x[1]['strike'])
                
                # Look for spreads (opposite position signs)
                for i in range(len(options)):
                    for j in range(i + 1, len(options)):
                        key1, pos1 = options[i]
                        key2, pos2 = options[j]
                        
                        # Check if positions are opposite (one long, one short)
                        if (pos1['position'] > 0 and pos2['position'] < 0) or (pos1['position'] < 0 and pos2['position'] > 0):
                            # Determine which is bought (long) and which is sold (short)
                            if pos1['position'] > 0:
                                bought_pos, sold_pos = pos1, pos2
                                bought_key, sold_key = key1, key2
                            else:
                                bought_pos, sold_pos = pos2, pos1
                                bought_key, sold_key = key2, key1
                            
                            # Format expiry
                            expiry_str = pos1['expiry']
                            if len(expiry_str) == 8:
                                # Convert YYYYMMDD to MMMyy format
                                year = expiry_str[:4]
                                month = expiry_str[4:6]
                                month_names = {
                                    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
                                    '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
                                    '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
                                }
                                expiry_formatted = f"{month_names.get(month, month)}{year[2:]}"
                            else:
                                expiry_formatted = expiry_str
                            
                            # Determine spread type
                            if pos1['right'] == 'P':  # Put spread
                                if bought_pos['strike'] < sold_pos['strike']:
                                    spread_type = "bull put"
                                else:
                                    spread_type = "bear put"
                            else:  # Call spread
                                if bought_pos['strike'] < sold_pos['strike']:
                                    spread_type = "bull call"
                                else:
                                    spread_type = "bear call"
                            
                            # Create financial instrument description
                            strikes = f"{int(sold_pos['strike'])}/{int(bought_pos['strike'])}"
                            financial_instrument = f"{pos1['symbol']} {expiry_formatted} {strikes} {spread_type}"
                            
                            # Create position descriptions
                            bought_description = f"{bought_pos['symbol']} {expiry_formatted} {int(bought_pos['strike'])} {bought_pos['right']}"
                            sold_description = f"{sold_pos['symbol']} {expiry_formatted} {int(sold_pos['strike'])} {sold_pos['right']}"
                            
                            # Calculate market values using best available price
                            bought_price, bought_source = self.get_best_price(bought_key)
                            sold_price, sold_source = self.get_best_price(sold_key)
                            
                            bought_market_value = 0.0
                            sold_market_value = 0.0
                            
                            if bought_price:
                                bought_market_value = float(bought_price) * float(bought_pos['position']) * 100.0
                            
                            if sold_price:
                                sold_market_value = float(sold_price) * float(sold_pos['position']) * 100.0
                            
                            whole_market_value = bought_market_value + sold_market_value
                            
                            # Calculate prices when actioned (from avgCost) - ensure all values are floats
                            bought_action_price = float(bought_pos['avgCost'])
                            sold_action_price = float(sold_pos['avgCost'])
                            
                            # Calculate whole position price when actioned
                            # This is the net credit/debit when the spread was opened
                            whole_action_price = (sold_action_price * abs(float(sold_pos['position'])) - 
                                                bought_action_price * abs(float(bought_pos['position'])))
                            
                            # Add data source info to the financial instrument name
                            price_source = ""
                            if bought_source == "close" or sold_source == "close":
                                price_source = " [using closing prices]"
                            elif bought_source == "live" or sold_source == "live":
                                price_source = " [using live prices]"
                            
                            spread_data.append({
                                'Financial Instruments': financial_instrument + price_source,
                                'Bought Position': bought_description,
                                'Sold Position': sold_description,
                                'Market Value of Whole Position': round(whole_market_value, 0),
                                'Market Value of Bought Position': round(bought_market_value, 0),
                                'Market Value of Sold Position': round(sold_market_value, 0),
                                'Price of Whole Position When Actioned': round(whole_action_price, 2),
                                'Price of Bought Position When Actioned': round(bought_action_price, 2),
                                'Price of Sold Position When Actioned': round(sold_action_price, 2)
                            })
        
        return pd.DataFrame(spread_data)
    
    def create_individual_positions_dataframe(self):
        """Create DataFrame for individual positions (non-spreads)"""
        individual_data = []
        
        for key, pos in self.positions.items():
            if pos['secType'] == 'OPT':
                # Format expiry
                expiry_str = pos['expiry']
                if len(expiry_str) == 8:
                    year = expiry_str[:4]
                    month = expiry_str[4:6]
                    month_names = {
                        '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
                        '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
                        '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
                    }
                    expiry_formatted = f"{month_names.get(month, month)}{year[2:]}"
                else:
                    expiry_formatted = expiry_str
                
                description = f"{pos['symbol']} {expiry_formatted} {int(pos['strike'])} {pos['right']}"
                
                # Calculate market value using best available price
                price, source = self.get_best_price(key)
                market_value = 0.0
                
                if price:
                    market_value = float(price) * float(pos['position']) * 100.0
                    if source == "close":
                        description += " [closing price]"
                    elif source == "live":
                        description += " [live price]"
                
                individual_data.append({
                    'Financial Instruments': description,
                    'Position': pos['position'],
                    'Market Value': round(market_value, 0),
                    'Price When Actioned': round(float(pos['avgCost']), 2)
                })
            
        return pd.DataFrame(individual_data)

In [ ]:
# Connect to IB Gateway
app = SimplePositionApp()

# IB Gateway typically runs on port 4001 (live) or 4002 (paper)
HOST = '127.0.0.1'
PORT = 4002  # Change to 4001 for live trading
CLIENT_ID = random.randint(1, 999)

print(f"Connecting to IB Gateway at {HOST}:{PORT} (Client ID: {CLIENT_ID})")
print("Make sure IB Gateway is running and API connections are enabled")

app.connect(HOST, PORT, CLIENT_ID)

# Start the message loop in a separate thread
thread = threading.Thread(target=app.run)
thread.daemon = True
thread.start()

time.sleep(2)  # Allow connection to establish

Connecting to IB Gateway at 127.0.0.1:4002 (Client ID: 329)
Make sure IB Gateway is running and API connections are enabled
Error 1754337957051: 2104
Error 1754337957057: 2106
Error 1754337957058: 2158


Found 6 positions
Error 1754337959133: 321
Error 1754337959233: 321
Error 1754337959333: 321
Error 1754337959433: 321
Error 1754337959533: 321
Error 1754337959633: 321
Error 1754337959733: 321
Error 1754337959833: 321
Error 1754337959933: 321
Error 1754337960033: 321
Error 1754337960034: 321
Error 1754337960133: 321


In [23]:
# Check connection and request positions
if app.isConnected():
    print("✅ Connected to IB Gateway")
    print("Requesting positions for account DU9233079...")
    
    app.reqPositions()
    
    # Wait for position data
    timeout = 20
    start_time = time.time()
    
    while not app.position_end and time.time() - start_time < timeout:
        time.sleep(0.5)
        print(".", end="", flush=True)
    print()
    
    # Wait a bit more for market data
    print("Getting market data...")
    time.sleep(5)
    
else:
    print("❌ Failed to connect to IB Gateway")
    print("Check that:")
    print("1. IB Gateway is running")
    print("2. API settings are enabled in IB Gateway")
    print("3. Port number is correct (4001 for live, 4002 for paper)")

✅ Connected to IB Gateway
Requesting positions for account DU9233079...
.
Getting market data...


In [24]:
# Display vertical spreads
if app.isConnected() and app.positions:
    spreads_df = app.create_spreads_dataframe()
    
    if not spreads_df.empty:
        print("\n=== VERTICAL SPREADS ===")
        display(spreads_df)
    else:
        print("\nNo vertical spreads found")
    
    # Also show individual positions
    individual_df = app.create_individual_positions_dataframe()
    
    if not individual_df.empty:
        print("\n=== ALL POSITIONS ===")
        display(individual_df)
else:
    print("No position data available")


=== VERTICAL SPREADS ===


,Financial Instruments,Bought Position,Sold Position,Market Value of Whole Position,Market Value of Bought Position,Market Value of Sold Position,Price of Whole Position When Actioned,Price of Bought Position When Actioned,Price of Sold Position When Actioned
0,UNH Aug25 240/230 bull put,UNH Aug25 230 P,UNH Aug25 240 P,0.0,0.0,0.0,128.67,185.66,314.34
1,UNH Aug25 240/230 bull put,UNH Aug25 230 P,UNH Aug25 240 P,0.0,0.0,0.0,133.67,310.66,444.34
2,AAPL Aug25 195/185 bull put,AAPL Aug25 185 P,AAPL Aug25 195 P,0.0,0.0,0.0,188.73,118.63,307.37



=== ALL POSITIONS ===


,Financial Instruments,Position,Market Value,Price When Actioned
0,UNH Aug25 240 P,-1,0.0,314.34
1,UNH Aug25 240 P,-1,0.0,444.34
2,AAPL Aug25 195 P,-1,0.0,307.37
3,UNH Aug25 230 P,1,0.0,310.66
4,UNH Aug25 230 P,1,0.0,185.66
5,AAPL Aug25 185 P,1,0.0,118.63


In [25]:
# Disconnect
if app.isConnected():
    app.disconnect()
    print("\n🔌 Disconnected from IB Gateway")


🔌 Disconnected from IB Gateway
